In [1]:
import itertools
from scipy.optimize import linear_sum_assignment
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
import os 
os.environ['CPLEX_STUDIO_DIR201'] = "/opt/ibm/ILOG/CPLEX_Studio_Community201"
os.environ['CPLEX_STUDIO_KEY'] = "xxxx"
from docplex.mp.model import Model
from docplex.mp.environment import Environment

In [3]:
permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]
permutations1 = ['12' + ''.join(x) for x in itertools.permutations(['3','4','5','6','7'], 5)]
permutations = permutations1 + permutations

In [4]:
N = len(permutations)
N

5160

In [5]:
MAX_COST = 3

In [6]:
p2i = {}
for i,p in enumerate(permutations):
    p2i[p] = i

In [7]:
permutations = np.array(permutations)
permutations_set = set(permutations)
#np.random.shuffle(permutations)

In [8]:
def get_cost(p1, p2):
    if p1 == p2:
        return 10000
    for i in range(1, MAX_COST+1):
        #if i == 2 and p1[0] == '1' and p1[2] == '2':
        #    return 10000
        if p1[i:] == p2[:-i]:
            return i 
    return 10000

p1, p2 = '1234567', '2345672'
get_cost(p1, p2)

1

In [9]:
def get_dmat(permutations):
    L = len(permutations)
    var_idx = []
    cost = {}
    for i,p1 in enumerate(tqdm(permutations)):
        for j,p2 in enumerate(permutations):
            cost_ij = get_cost(p1, p2)
            if cost_ij <= MAX_COST:
                var_idx.append((i, j))
                cost[(i,j)] = cost_ij
    return var_idx, cost

var_idx, all_cost = get_dmat(permutations)

len(var_idx)

100%|██████████████████████████████████████| 5160/5160 [00:38<00:00, 135.47it/s]


47520

In [10]:
def get_pair_idx(var_idx):
    firsts = {}
    lasts = {}
    for i,j in var_idx:
        if not firsts.get(j, False):
            firsts[j] = [i]
        else:
            firsts[j].append(i)
        if not lasts.get(i, False):
            lasts[i] = [j]
        else:
            lasts[i].append(j)
    return firsts, lasts


firsts, lasts = get_pair_idx(var_idx)

In [11]:
len(firsts[0]), len(lasts[0])

(9, 9)

In [12]:
env = Environment()
env.print_information()

mdl = Model("santa")
next_perm = mdl.binary_var_dict(var_idx)
start_perm = mdl.binary_var_list(range(N), name='start')
end_perm = mdl.binary_var_list(range(N), name='end')
class_perm = mdl.continuous_var_matrix(range(N), range(3), lb=0, ub=1, name='class')
class_perm_binary = mdl.binary_var_matrix(range(240), range(3), name='class_binary')
mdl.print_information()

* system is: Linux 64bit
* Python version 3.8.12, located at: /home/jpuget/miniconda3/envs/cplex201/bin/python
* docplex is present, version is 2.18.200
* CPLEX library is present, version is 20.1.0.0, located at: /home/jpuget/miniconda3/envs/cplex201/lib/python3.8/site-packages
* pandas is present, version is 1.3.4
Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [13]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i in firsts[j]) + start_perm[j] == 1
    for j in range(N))

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for j in lasts[i]) + end_perm[i] == 1
    for i in range(N))

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 10320
   - linear=10320
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [14]:
mdl.add_constraint(mdl.sum(start_perm) == mdl.sum(end_perm))
mdl.add_constraint(mdl.sum(start_perm) == 90)
#mdl.add_constraints(start_perm[i] <= start_perm[1+120] for i in range(120))

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 10322
   - linear=10322
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [15]:
# from model_045 solution

mdl.add_constraints(start_perm[i] == 0 for i,p in enumerate(permutations) if not p.startswith('12'))
mdl.add_constraints(end_perm[i] == 0 for i,p in enumerate(permutations) if (p[0] != '1' and p[2] != '2'))

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 18962
   - linear=18962
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [16]:
# next_perm[(i,j)] == 1 => (class_perm[i] == class_perm[j])

mdl.add_constraints(
    class_perm[(i, k)] - class_perm[(j, k)] <= (1 - next_perm[(i,j)])
    for i,j in var_idx
    for k in range(3)
)

mdl.add_constraints(
    class_perm[(j, k)] - class_perm[(i, k)] <= (1 - next_perm[(i,j)])
    for i,j in var_idx
    for k in range(3)
)

mdl.add_constraints(
    class_perm_binary[(i+120, k)] + class_perm_binary[(i, k)] <= 1
    for i in range(120)
    for k in range(3)
)

mdl.add_constraints(
    class_perm[(i,k)] == class_perm_binary[(i,k)]
    for i in range(240)
    for k in range(3)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for k in range(3)) == 1
    for i in range(240)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for i in range(N)) == N // 3
    for k in range(3)
)

mdl.add_constraints(
    mdl.sum(class_perm[(i,k)] for i in range(240)) == 240 // 3
    for k in range(3)
)

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 305408
   - linear=305408
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [17]:
def rot(s, i=1):
    if i <= 0:
        return s
    return s[i:] + s[:i]

def get_canon(s):
    best = s
    for i in range(len(s)-1):
        s = rot(s)
        if s[0] < best[0]:
            best = s
    return best

In [18]:
def get_1_cycle(i, s):
    cycle = [rot(s, i) for i in range(len(s))]
    cycle = [p2i[p] for p in cycle]
    cycle[0] = i
    return cycle
    
cycles1 = []
for i,p in enumerate(permutations):
    if p != get_canon(p):
        continue
    cycles1.append(get_1_cycle(i, p))

def get_alternate(i):
    if i < 120:
        return i + 120
    elif i < 240:
        return i - 120
    return i

def get_inner(cycle):
    inner = []
    for j in cycle:
        inner.extend([(i, j) for i in firsts[j] if i not in cycle and get_alternate(i) not in cycle])
    return inner

def get_outer(cycle):
    outer = []
    for i in cycle:
        outer.extend([(i, j) for j in lasts[i] if j not in cycle and get_alternate(j) not in cycle])
    return outer

var_idx_set = set(var_idx)

def get_cycle_edges(cycle):
    edges = [(i,j) for i,j in itertools.permutations(cycle, 2) if (i,j) in var_idx_set]
    return edges
    

In [19]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) + mdl.sum(start_perm[j] for j in cycle) >= 1
    for cycle in cycles1
)

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) + mdl.sum(end_perm[i] for i in cycle) >= 1
    for cycle in cycles1
)

#mdl.add_constraints(
#    mdl.sum(all_cost[(i,j)] * next_perm[(i,j)] for i,j in get_cycle_edges(cycle)) <= len(cycle) - 1
#    for cycle in cycles1
#)

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 307088
   - linear=307088
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [20]:
def remove_start(s, start):
    idx = s.index(start)
    s = s[idx:] + s[:idx]
    return s[0] + get_canon(s[1:])

def get_2_cycle(p, start):
    p = remove_start(p, start)
    p1 = p[1:]
    inner = [rot(p1, i) for i in range(len(p1))]
    cycle = [rot(q+start, j) for q in inner for j in range(len(p))]
    #cycle1 = [get_1_cycle(q+start) for q in inner]
    #cycle = get_canon(cycle)
    cycle = [p2i[p] for p in cycle]
    return p, cycle

cycles2 = {}

for p in tqdm(permutations[120:]):
    for start in p:
        p1 = remove_start(p, start)
        if cycles2.get(p1, None) is not None:
            continue
        p1, cycle = get_2_cycle(p, start)
        cycles2[p1] = cycle

cycles2['2134567']

len(cycles2)

def get_alternate_2cycles(cycle):
    new_cycle = []
    duplicate = False
    for i in cycle:
        ii = get_alternate(i)
        if i != ii:
            duplicate = True
        new_cycle.append(ii)
    if duplicate:
        return [cycle, new_cycle]
    else:
        return [cycle]

all_cycles2 = [cycle for v in cycles2.values() for cycle in get_alternate_2cycles(v)]
len(all_cycles2)

100%|████████████████████████████████████| 5040/5040 [00:00<00:00, 33963.61it/s]


1200

In [21]:
sorted([p for p in [permutations[i] for i in cycles2['3124567']] if p.startswith('12')])

['1234567', '1243567', '1245367', '1245637', '1245673']

In [22]:
mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_inner(cycle)) + mdl.sum(start_perm[j] for j in cycle) >= 1
    for cycle in all_cycles2
)

mdl.add_constraints(
    mdl.sum(next_perm[(i,j)] for i,j in get_outer(cycle)) + mdl.sum(end_perm[i] for i in cycle) >= 1
    for cycle in all_cycles2
)

mdl.print_information()

Model: santa
 - number of variables: 74040
   - binary=58560, integer=0, continuous=15480
 - number of constraints: 309488
   - linear=309488
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [23]:
N3 = N // 3
#N3 = 108

u = mdl.continuous_var_list(range(N), lb=0, ub=N3-1, name='u')

mdl.add_constraints(
    u[i] - u[j] + N3 * next_perm[(i,j)] <= N3 - 1
    for i,j in var_idx 
)

mdl.add_constraints(
    u[i] <= (N3 - 1) * (1 - start_perm[i])
    for i in range(N)
)

mdl.add_constraints(
    u[i] >= (1 - start_perm[i])
    for i in range(N)
)

mdl.print_information()

Model: santa
 - number of variables: 79200
   - binary=58560, integer=0, continuous=20640
 - number of constraints: 367328
   - linear=367328
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [24]:
s0 = '21111112111111211111121111112111111'
#s1 = '111121' + s0
def get_string(start, code):
    string = start
    seq = [p2i[start]]
    for c in code:
        if c == '1':
            start = rot(start)
            string = string + start[-1]
            seq.append(p2i[start])
        elif c == '2':
            start = rot(start, 2)
            start = start[:5] + start[-1] + start[-2]
            string = string + start[-2:]
            seq.append(p2i[start])
        elif c == '3':
            start = rot(start, 3)            
            start = start[:4] + start[-1] + start[-3] + start[-2]
            string = string + start[-3:]
            seq.append(p2i[start])
    return seq, string

for k in range(120):
    seq, _ = get_string(permutations[k], s0)
    mdl.add_constraints(
        next_perm[(i,j)] == 1 for i,j in zip(seq[:-1], seq[1:])
    )

mdl.print_information()

Model: santa
 - number of variables: 79200
   - binary=58560, integer=0, continuous=20640
 - number of constraints: 371528
   - linear=371528
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [25]:
objective = mdl.sum(next_perm[idx] * all_cost[idx] for idx in var_idx) + 6.999 * mdl.sum(start_perm)
                          
# Set objective function
mdl.minimize(objective)

mdl.print_information()

mdl.parameters.mip.tolerances.mipgap = 0.00
mdl.parameters.timelimit = 100000 
mdl.parameters.threads = 20
#mdl.parameters.mip.cuts.gomory = -1
mdl.parameters.mip.tolerances.uppercutoff = 7320 - 7*120 + 1e-3

mdl.solve(log_output=True)
mdl.report()
  

Model: santa
 - number of variables: 79200
   - binary=58560, integer=0, continuous=20640
 - number of constraints: 371528
   - linear=371528
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Checking license ...
License found. [0.04 s]
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 20
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               100000
CPXPARAM_MIP_Tolerances_UpperCutoff              6480.0010000000002
CPXPARAM_MIP_Tolerances_MIPGap                   0
Tried aggregator 6 times.
MIP Presolve eliminated 303000 rows and 50520 columns.
MIP Presolve modified 83520 coefficients.
Aggregator did 17520 substitutions.
Reduced MIP has 51008 rows, 11160 columns, and 199200 nonzeros.
Reduced MIP has 8040 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.70 sec. (652.36 ticks)
Probing time = 0.

In [26]:
sol = mdl.solution
sol

s = sol.get_value_dict(next_perm)
s
s_start = sol.get_value_list(start_perm)
s_end = sol.get_value_list(end_perm)
s_u = sol.get_value_list(u)
s_class = sol.get_value_dict(class_perm)


In [27]:
s_cost = {}
s_cost1 = {}
s_sol = {}
for k,v in s.items():
    if v > 0:
        s_cost[k] = all_cost[k] * v
        s_cost1[k] = all_cost[k] 
        s_sol[k] = v

s_cost



C = Counter(s_cost.values())
C1 = Counter(s_cost1.values())
Cs = Counter(s_sol.values())
C, C1, Cs, Counter(s_start), Counter(s_end), Counter(s_class.values())

(Counter({1.0: 4320, 2.0: 720, 3.0: 30}),
 Counter({1: 4320, 2: 720, 3: 30}),
 Counter({1.0: 5070}),
 Counter({1.0: 90, 0: 5070}),
 Counter({0: 5070, 1.0: 90}),
 Counter({1.0: 5021,
          0: 10320,
          0.9999999999999999: 42,
          0.9999999999999805: 7,
          0.9999999999999792: 3,
          0.9999999999999859: 15,
          1.0000000000000013: 36,
          0.9999999999999878: 36}))

In [28]:
for (i,j), v in s_cost1.items():
    if v==3:
        print(permutations[i], permutations[j])

1423576 3576124
1423675 3675124
1423756 3756124
1423765 3765124
1425376 5376124
1425736 5736124
1426375 6375124
1427356 7356124
1427536 7536124
1523467 3467125
1523647 3647125
1526347 6347125
1624357 4357126
1624537 4537126
1624735 4735126
1625437 5437126
1625473 5473126
1625743 5743126
1627345 7345126
1627435 7435126
1627543 7543126
1723564 3564127
1723654 3654127
1724563 4563127
1724653 4653127
1725364 5364127
1725634 5634127
1726354 6354127
1726453 6453127
1726534 6534127


In [29]:
p_class = [int(np.round(s_class[(i, 1)] + 2 * s_class[(i, 2)]) )for i in range(N)]
Counter(p_class)

Counter({0: 1720, 2: 1720, 1: 1720})

In [30]:
Counter(p_class[:240])

Counter({0: 80, 2: 80, 1: 80})

In [31]:
costs = np.zeros(3)
for (i,j), c in s_cost1.items():
    if (p_class[i] != p_class[j]):
        print(p_class[i], p_class[j])
    costs[(p_class[i])] += c
for i in range(N):
    if s_start[i] > 0:
        costs[(p_class[i])] += 7
costs

array([2160., 2160., 2160.])

In [32]:
def get_cycle(start, s_sol, s_cost, s_start, s_end):
    if s_start[start]:
        cycle_cost = 7
    else:
        cycle_cost = 0
    cycle = [start]
    prev = start
    while True:
        if s_end[prev]:
            return cycle, cycle_cost
        candidates0 = []
        for j in lasts[prev]:
            if s_sol.get((prev, j), 0) > 0:
                candidates0.append(j)
        candidates1 = [j for j in candidates0 if j not in cycle]
        if len(candidates1) > 0:
            candidates = candidates1
        else:
            return cycle, cycle_cost
        best_cost = 10000
        best_j = -1
        for j in candidates:
            cost = s_cost[(prev, j)]
            if cost < best_cost:
                best_cost = cost
                best_j = j
        cycle.append(best_j)
        cycle_cost = cycle_cost + s_cost[(prev, best_j)]
        s_sol[(prev, best_j)] = s_sol[(prev, best_j)] - 1
        prev = best_j    
    return cycle, cycle_cost
        

In [33]:
all_cycles = [[], [], []]
remaining = 1e6
mandatory_start = [i for i in range(N) if s_start[i]]
s_sol1 = s_sol.copy()

for k in mandatory_start:
    cycle, cost =  get_cycle(k, s_sol1, s_cost1, s_start, s_end)    
    Cs = Counter(s_sol1.values())
    remaining = np.sum([k*v for k,v in Cs.items()])    
    all_cycles[p_class[k]].append(cycle)
    print(len(cycle), cost, remaining, (len(cycle) - 1) // 36, (len(cycle) - 1) % 36, Cs)
    if remaining == 0:
        break

Cs = Counter(s_sol1.values())
Cs

42 54 5029.0 1 5 Counter({1.0: 5029, 0.0: 41})
82 101 4948.0 2 9 Counter({1.0: 4948, 0.0: 122})
42 54 4907.0 1 5 Counter({1.0: 4907, 0.0: 163})
42 54 4866.0 1 5 Counter({1.0: 4866, 0.0: 204})
42 54 4825.0 1 5 Counter({1.0: 4825, 0.0: 245})
152 183 4674.0 4 7 Counter({1.0: 4674, 0.0: 396})
48 61 4627.0 1 11 Counter({1.0: 4627, 0.0: 443})
162 195 4466.0 4 17 Counter({1.0: 4466, 0.0: 604})
42 54 4425.0 1 5 Counter({1.0: 4425, 0.0: 645})
94 115 4332.0 2 21 Counter({1.0: 4332, 0.0: 738})
82 101 4251.0 2 9 Counter({1.0: 4251, 0.0: 819})
88 108 4164.0 2 15 Counter({1.0: 4164, 0.0: 906})
42 54 4123.0 1 5 Counter({1.0: 4123, 0.0: 947})
78 96 4046.0 2 5 Counter({1.0: 4046, 0.0: 1024})
42 54 4005.0 1 5 Counter({1.0: 4005, 0.0: 1065})
42 54 3964.0 1 5 Counter({1.0: 3964, 0.0: 1106})
134 162 3831.0 3 25 Counter({1.0: 3831, 0.0: 1239})
82 101 3750.0 2 9 Counter({1.0: 3750, 0.0: 1320})
54 68 3697.0 1 17 Counter({1.0: 3697, 0.0: 1373})
42 54 3656.0 1 5 Counter({1.0: 3656, 0.0: 1414})
88 108 3569.0 2 1

Counter({0.0: 5070})

In [34]:
[np.sum([len(cycle) for cycle in all_cycles[i]]) for i in range(3)]

[1720, 1720, 1720]

In [35]:
def get_concat(perms):
    string = permutations[perms[0]]
    
    for j in perms[1:]:
        p = permutations[j]
        if p == string[-7:]:
            string = string + p
            continue
        for i in range(1, 8):
            if string[i-7:] == p[:-i]:
                break
        if i < 7:
            string = string + p[-i:]
        else:
            string = string + p
    return string

In [36]:
cats = [[get_concat(cycle) for cycle in all_cycles[i]] for i in range(3)]
[len(cat) for cat in cats]

[30, 30, 30]

In [37]:
[np.sum([len(cat) for cat in catsi]) for catsi in cats]

[2160, 2160, 2160]

In [38]:
2160 + 40*7

2440

In [39]:
cats[0]

['123456712346572134657231465723416572346157234651723465',
 '123476512374652137465231746523714652374165237461523746',
 '123567412356472135647231564723516472356147235641723564127356412375641235764213576423157642351764235716423576142357612435761234576123547612357462135746231574623517462357146235741623574',
 '123745612375462137546231754623715462375146237541623754',
 '124367512346751236475123674512367541236574123654712364572136457231645723614572364157236451723645',
 '124375612347562134756231475623417562347156234751623475',
 '124536712453762145376241537624513762453176245371624537126453712465371245637214563724156372451637245613724563172456312745632174563271456327415632745163274561327456',
 '124635712436572143657241365724316572436157243651724365',
 '124753612745362174536271453627415362745136274531627453',
 '124756312457632145763241576324517632457163245761324576',
 '124763512473652147365241736524713652473165247361524736',
 '125346712354672135467231546723514672354167235461723546',
 '12537461253

In [40]:
def get_perms(cat):
    perms = [cat[i:i+7] for i in range(len(cat))]
    perms = [p for p in perms if p in permutations]
    return perms

all_perms = [p for catsi in cats for cat in catsi for p in get_perms(cat)]

In [41]:
def get_string(cycles):
    s = ''.join([get_concat(cycle) for cycle in cycles])
    used = []
    for cycle in cycles:
        cycle = [permutations[i] for i in cycle]
        used.extend(p for p in permutations1 if p in cycle)
    s = s + ''.join([p for p in permutations1 if p not in used])
    return s

In [42]:
strings = [get_string(cycles) for cycles in all_cycles]
[len(s) for s in strings]

[2440, 2440, 2440]

In [43]:
perms = [get_perms(s) for s in strings]
all_perms = [p for perm in perms for p in perm]

In [44]:
len(all_perms)

5915

In [45]:
for p in permutations:
    if not p in all_perms:
        print(p)

In [46]:
for p in permutations1:
    for perm in perms:
        if not p in perm:
            print(p)

In [47]:
def finalize(strings):
    replace_dict = {
        "1": '🎅', 
        "2": '🤶', 
        "3": '🦌', 
        "4": '🧝', 
        "5": '🎄', 
        "6": '🎁', 
        "7": '🎀', 
        "*": '🌟',
    }
    ans = strings.copy()
    for i in range(3):
        for k,v in replace_dict.items():
            ans[i] = ans[i].replace(k, v)
    return ans

In [48]:
ans = finalize(strings)

In [49]:
ans

['🎅🤶🦌🧝🎄🎁🎀🎅🤶🦌🧝🎁🎄🎀🤶🎅🦌🧝🎁🎄🎀🤶🦌🎅🧝🎁🎄🎀🤶🦌🧝🎅🎁🎄🎀🤶🦌🧝🎁🎅🎄🎀🤶🦌🧝🎁🎄🎅🎀🤶🦌🧝🎁🎄🎅🤶🦌🧝🎀🎁🎄🎅🤶🦌🎀🧝🎁🎄🤶🎅🦌🎀🧝🎁🎄🤶🦌🎅🎀🧝🎁🎄🤶🦌🎀🎅🧝🎁🎄🤶🦌🎀🧝🎅🎁🎄🤶🦌🎀🧝🎁🎅🎄🤶🦌🎀🧝🎁🎅🤶🦌🎄🎁🎀🧝🎅🤶🦌🎄🎁🧝🎀🤶🎅🦌🎄🎁🧝🎀🤶🦌🎅🎄🎁🧝🎀🤶🦌🎄🎅🎁🧝🎀🤶🦌🎄🎁🎅🧝🎀🤶🦌🎄🎁🧝🎅🎀🤶🦌🎄🎁🧝🎅🤶🎀🦌🎄🎁🧝🎅🤶🦌🎀🎄🎁🧝🎅🤶🦌🎄🎀🎁🧝🤶🎅🦌🎄🎀🎁🧝🤶🦌🎅🎄🎀🎁🧝🤶🦌🎄🎅🎀🎁🧝🤶🦌🎄🎀🎅🎁🧝🤶🦌🎄🎀🎁🎅🧝🤶🦌🎄🎀🎁🎅🤶🧝🦌🎄🎀🎁🎅🤶🦌🧝🎄🎀🎁🎅🤶🦌🎄🧝🎀🎁🎅🤶🦌🎄🎀🧝🎁🤶🎅🦌🎄🎀🧝🎁🤶🦌🎅🎄🎀🧝🎁🤶🦌🎄🎅🎀🧝🎁🤶🦌🎄🎀🎅🧝🎁🤶🦌🎄🎀🧝🎅🎁🤶🦌🎄🎀🧝🎅🤶🦌🎀🧝🎄🎁🎅🤶🦌🎀🎄🧝🎁🤶🎅🦌🎀🎄🧝🎁🤶🦌🎅🎀🎄🧝🎁🤶🦌🎀🎅🎄🧝🎁🤶🦌🎀🎄🎅🧝🎁🤶🦌🎀🎄🧝🎅🎁🤶🦌🎀🎄🧝🎅🤶🧝🦌🎁🎀🎄🎅🤶🦌🧝🎁🎀🎄🎅🤶🦌🎁🧝🎀🎄🎅🤶🦌🎁🎀🧝🎄🎅🤶🦌🎁🎀🎄🧝🎅🤶🦌🎁🎄🎀🧝🎅🤶🦌🎁🎄🧝🎀🎅🤶🦌🎁🧝🎄🎀🤶🎅🦌🎁🧝🎄🎀🤶🦌🎅🎁🧝🎄🎀🤶🦌🎁🎅🧝🎄🎀🤶🦌🎁🧝🎅🎄🎀🤶🦌🎁🧝🎄🎅🎀🤶🦌🎁🧝🎄🎅🤶🧝🦌🎀🎄🎁🎅🤶🦌🧝🎀🎄🎁🤶🎅🦌🧝🎀🎄🎁🤶🦌🎅🧝🎀🎄🎁🤶🦌🧝🎅🎀🎄🎁🤶🦌🧝🎀🎅🎄🎁🤶🦌🧝🎀🎄🎅🎁🤶🦌🧝🎀🎄🎅🤶🧝🎄🦌🎁🎀🎅🤶🧝🎄🦌🎀🎁🤶🎅🧝🎄🦌🎀🎁🤶🧝🎅🎄🦌🎀🎁🤶🧝🎄🎅🦌🎀🎁🤶🧝🎄🦌🎅🎀🎁🤶🧝🎄🦌🎀🎅🎁🤶🧝🎄🦌🎀🎅🤶🎁🧝🎄🦌🎀🎅🤶🧝🎁🎄🦌🎀🎅🤶🧝🎄🎁🦌🎀🤶🎅🧝🎄🎁🦌🎀🤶🧝🎅🎄🎁🦌🎀🤶🧝🎄🎅🎁🦌🎀🤶🧝🎄🎁🎅🦌🎀🤶🧝🎄🎁🦌🎅🎀🤶🧝🎄🎁🦌🎅🤶🎀🧝🎄🎁🦌🤶🎅🎀🧝🎄🎁🦌🤶🎀🎅🧝🎄🎁🦌🤶🎀🧝🎅🎄🎁🦌🤶🎀🧝🎄🎅🎁🦌🤶🎀🧝🎄🎁🎅🦌🤶🎀🧝🎄🎁🎅🤶🧝🎁🦌🎄🎀🎅🤶🧝🦌🎁🎄🎀🤶🎅🧝🦌🎁🎄🎀🤶🧝🎅🦌🎁🎄🎀🤶🧝🦌🎅🎁🎄🎀🤶🧝🦌🎁🎅🎄🎀🤶🧝🦌🎁🎄🎅🎀🤶🧝🦌🎁🎄🎅🤶🧝🎀🎄🦌🎁🎅🤶🎀🧝🎄🦌🎁🤶🎅🎀🧝🎄🦌🎁🤶🎀🎅🧝🎄🦌🎁🤶🎀🧝🎅🎄🦌🎁🤶🎀🧝🎄🎅🦌🎁🤶🎀🧝🎄🦌🎅🎁🤶🎀🧝🎄🦌🎅🤶🧝🎀🎄🎁🦌🎅🤶🧝🎄🎀🎁🦌🤶🎅🧝🎄🎀🎁🦌🤶🧝🎅🎄🎀🎁🦌🤶🧝🎄🎅🎀🎁🦌🤶🧝🎄🎀🎅🎁🦌🤶🧝🎄🎀🎁🎅🦌🤶🧝🎄🎀🎁🎅🤶🧝🎀🎁🦌🎄🎅🤶🧝🎀🦌🎁🎄🤶🎅🧝🎀🦌🎁🎄🤶🧝🎅🎀🦌🎁🎄🤶🧝🎀🎅🦌🎁🎄🤶🧝🎀🦌🎅🎁🎄🤶🧝🎀🦌🎁🎅🎄🤶🧝🎀🦌🎁🎅🤶🎄🦌🧝🎁🎀🎅🤶🦌🎄🧝🎁🎀🤶🎅🦌🎄🧝🎁🎀🤶🦌🎅🎄🧝🎁🎀🤶🦌🎄🎅🧝🎁🎀🤶🦌🎄🧝🎅🎁🎀🤶🦌🎄🧝🎁🎅🎀🤶🦌🎄🧝🎁🎅🤶🎄🦌🎀🧝🎁🎅🤶🎄🦌🎀🎁🧝🎅🤶🎄🦌🎁🎀🧝🤶🎅🎄🦌🎁🎀🧝🤶🎄🎅🦌🎁🎀🧝🤶🎄🦌🎅🎁🎀🧝🤶🎄🦌🎁🎅🎀🧝🤶🎄🦌🎁🎀🎅🧝🤶🎄🦌🎁🎀🎅🤶🎁🦌🎄🎀🧝🎅🤶🎁

In [50]:
import pandas as pd
sub = pd.DataFrame()
sub['schedule'] = ans
sub.to_csv('../subs/submission_2440_final.csv',index=False)
sub.head()

,schedule
0,🎅🤶🦌🧝🎄🎁🎀🎅🤶🦌🧝🎁🎄🎀🤶🎅🦌🧝🎁🎄🎀🤶🦌🎅🧝🎁🎄🎀🤶🦌🧝🎅🎁🎄🎀🤶🦌🧝🎁🎅🎄🎀🤶🦌🧝🎁...
1,🎅🤶🦌🎄🧝🎁🎀🎅🤶🦌🧝🎄🎁🎀🤶🎅🦌🧝🎄🎁🎀🤶🦌🎅🧝🎄🎁🎀🤶🦌🧝🎅🎄🎁🎀🤶🦌🧝🎄🎅🎁🎀🤶🦌🧝🎄...
2,🎅🤶🦌🧝🎁🎄🎀🎅🤶🦌🧝🎁🎀🎄🤶🎅🦌🧝🎁🎀🎄🤶🦌🎅🧝🎁🎀🎄🤶🦌🧝🎅🎁🎀🎄🤶🦌🧝🎁🎅🎀🎄🤶🦌🧝🎁...
